In [8]:
import tkinter as tk
from tkinter import filedialog
import csv, os
from PIL import Image, ImageTk
import numpy as np
import pandas as pd
import tkinter.messagebox as messagebox

red_points_array = None
green_points_array = None

class PointGridApp:
    def __init__(self, master, rows=10, columns=10, cell_size=40, kernel_size=1):
        self.master = master
        self.rows = rows
        self.columns = columns
        self.cell_size = cell_size
        self.kernel_size = kernel_size  # Size of the selection/deselection area
        self.vertex_radius = 3  # Radius of the vertex circles
        self.red_points = {(col, row) for row in range(1, self.rows) for col in range(1, self.columns)}
        self.green_points = set()
        self.red_points_array = None  
        self.mode = "select"  # Mode can be "select" or "erase"
        self.line_start = None
        self.current_line = None
        self.lines = []   
        self.color_mode = "red"
        
        self.create_widgets()

    def create_widgets(self):
        self.canvas = tk.Canvas(self.master, width=self.columns * self.cell_size, 
                                height=self.rows * self.cell_size)
        self.canvas.bind("<Button-1>", self.toggle_point)
        self.canvas.bind("<B1-Motion>", self.drag_toggle)
        self.canvas.grid(row=0, column=0, columnspan=4)

        self.filename_entry = tk.Entry(self.master, text="Filename")
        self.filename_entry.grid(row=2, column=0)
        
        self.export_button = tk.Button(self.master, text="Export", command=self.export_points)
        self.export_button.grid(row=2, column=1)
        
        self.clear_button = tk.Button(self.master, text="Clear All", command=self.clear_all)
        self.clear_button.grid(row=2, column=2)
        
        self.message_label = tk.Label(self.master, text="")
        self.message_label.grid(row=3, column=0, columnspan=4)

        self.toggle_mode_button = tk.Button(self.master, text="Selecting", command=self.toggle_mode)
        self.toggle_mode_button.grid(row=1, column=1)

        self.toggle_color_mode_button = tk.Button(self.master, text="Red", command=self.toggle_color_mode)
        self.toggle_color_mode_button.grid(row=1, column=2)
        
        self.line_mode_button = tk.Button(self.master, text="Point", command=self.toggle_line_mode)
        self.line_mode_button.grid(row=1, column=3)

        self.draw_grid()
        self.mark_all_vertices()
        

    def draw_grid(self):
        for i in range(self.rows):
            for j in range(self.columns):
                self.canvas.create_rectangle(j * self.cell_size, i * self.cell_size, 
                                             (j+1) * self.cell_size, (i+1) * self.cell_size, outline="gray")

    def mark_all_vertices(self):
        for point in self.red_points:
            self.draw_point(*point, "red")

    def toggle_point(self, event):
        col = round(event.x / self.cell_size)
        row = round(event.y / self.cell_size)
        self.update_point(col, row, self.mode)

    def drag_toggle(self, event):
        if self.mode == "draw_line":
            self.draw_line(event)
        else:
            col = round(event.x / self.cell_size)
            row = round(event.y / self.cell_size)
            for dx in range(-self.kernel_size, self.kernel_size + 1):
                for dy in range(-self.kernel_size, self.kernel_size + 1):
                    self.update_point(col + dx, row + dy, self.mode)

    def update_point(self, col, row, mode):
        if 0 <= col <= self.columns and 0 <= row <= self.rows:
            if mode == "select":
                self.red_points.add((col, row))
            elif mode == "erase" and (col, row) in self.red_points:
                self.red_points.remove((col, row))
            self.redraw_points()

    def draw_point(self, col, row, color):
        x = col * self.cell_size
        y = row * self.cell_size
        self.canvas.create_oval(x - self.vertex_radius, y - self.vertex_radius,
                                x + self.vertex_radius, y + self.vertex_radius,
                                fill=color, outline=color)

    def toggle_color_mode(self):
        if self.color_mode == "red":
            self.color_mode = "green"
            self.toggle_color_mode_button.config(text="Green")
        else:
            self.color_mode = "red"
            self.toggle_color_mode_button.config(text="Red")

    def update_point(self, col, row, mode):
        if 0 <= col <= self.columns and 0 <= row <= self.rows:
            point = (col, row)
            if mode == "select":
                if self.color_mode == "red":
                    self.red_points.add(point)
                elif self.color_mode == "green":
                    self.green_points.add(point)
            elif mode == "erase":
                self.red_points.discard(point)
                self.green_points.discard(point)
            self.redraw_points()

    def redraw_points(self):
        self.canvas.delete("all")
        self.draw_grid()
        for point in self.red_points:
            self.draw_point(*point, "red")
        for point in self.green_points:
            self.draw_point(*point, "green")

    def toggle_mode(self):
        if self.mode == "select":
            self.mode = "erase"
            self.toggle_mode_button.config(text="Erasing")
        else:
            self.mode = "select"
            self.toggle_mode_button.config(text="Selecting")

    def toggle_line_mode(self):
        if self.mode != "draw_line":
            self.mode = "draw_line"
            self.canvas.bind("<Button-1>", self.start_line)
            self.canvas.bind("<B1-Motion>", self.draw_line)
            self.canvas.bind("<ButtonRelease-1>", self.end_line)
            self.line_mode_button.config(text="Line")
        else:
            self.mode = "select"
            self.canvas.bind("<Button-1>", self.toggle_point)
            self.canvas.bind("<B1-Motion>", self.drag_toggle)
            self.canvas.unbind("<ButtonRelease-1>")
            self.line_mode_button.config(text="Point")

    def start_line(self, event):
        self.line_start = self.get_nearest_point(event.x, event.y)
        self.current_line = None

    def draw_line(self, event):
        end_point = self.get_nearest_point(event.x, event.y)
        if self.line_start:
            self.current_line = self.canvas.create_line(
                self.line_start[0] * self.cell_size, self.line_start[1] * self.cell_size,
                end_point[0] * self.cell_size, end_point[1] * self.cell_size, fill="blue", width=3)
            self.lines.append((self.line_start, end_point))
            self.line_start = end_point

    def end_line(self, event):
        self.line_start = None
        self.current_line = None
        for line in self.lines:
            self.canvas.create_line(
                line[0][0] * self.cell_size, line[0][1] * self.cell_size,
                line[1][0] * self.cell_size, line[1][1] * self.cell_size, fill="blue", width=3)

    def get_nearest_point(self, x, y):
        col = round(x / self.cell_size)
        row = round(y / self.cell_size)
        return (col, row)
    
    def clear_all(self):
        self.red_points = set()
        self.green_points = set()
        self.lines = []
        self.canvas.delete("all")
        self.draw_grid()
        
    def capture_canvas(self):
        # Create an empty image with the same dimensions as the canvas
        screenshot = Image.new("RGB", (self.canvas.winfo_width(), self.canvas.winfo_height()))

        # Use the canvas' postscript method to draw its contents onto the image
        self.canvas.postscript(file="tmp.ps", colormode="color")
        tmp_image = Image.open("tmp.ps")
        screenshot.paste(tmp_image, (0, 0))

        return screenshot
    
    def export_points(self):
        if (len(self.red_points) > 0) or (len(self.green_points) > 0):
            filename = self.filename_entry.get()
            if not filename:
                filename = "combined_points"

            if not filename.lower().endswith(".csv"):
                filename = filename + '.csv'

            # Create separate DataFrames for red and green points
            red_df = pd.DataFrame({
                'x_r': [point[0] for point in self.red_points],
                'y_r': [point[1] for point in self.red_points]
            })

            green_df = pd.DataFrame({
                'x_g': [point[0] for point in self.green_points],
                'y_g': [point[1] for point in self.green_points]
            })
            combined_df = pd.concat([red_df, green_df], axis=1)
            combined_df.to_csv(filename, index=False)
            
            screenshot = self.capture_canvas()
            screenshot_filename = f"{filename[:-4]}.png"
            screenshot.save(screenshot_filename, "PNG")
            
            self.message_label.config(text=f"Data exported to {os.getcwd()+'/'+filename}")
        else:
            self.message_label.config(text="Error!!! No points selected")
root = tk.Tk()
app = PointGridApp(root)
root.mainloop()

